# Calculate matched pairs for a custom peptide library

For this tutorial, an example will be illustrated to extract matched pairs information from a SAR dataset, which can be obtained from local csv files or by generating libraries with PepFuNN functionalities. These methods have been configured to be flexible around the nature of the molecules in terms of the number of reference peptides included. Here we provide some examples depending on the question to answer

Before starting, the pepfunn package can be installed as follows:

In [ ]:
!pip install git+https://github.com/novonordisk-research/pepfunn.git

Another way is to download the gihub code and running locally in the same folder: `pip install -e .`

### 1. Using a dataset with a reference peptide

As an example, we will generate a library and calculate the molecular weigth as a proxy of an activity or property to be improved. The goal is to check all the pairs and the mutations responsible to increase or decrease the property based on a reference peptide. First we generate the library:

In [4]:
import pandas as pd
from pepfunn.library import Library
from rdkit import Chem
from rdkit.Chem import Descriptors

libtest=Library(population_size=100, mode='scanning', mode_scan='all', seeds=['FNCREWCWN'], pairs=[(2,'A'),(3,'L'),(5,'R'),(7,'M')], positions=[2,3,5,7], from_child=True, verbose=False)
    
dict_comparison={'ID':[], 'Sequence':[], 'MW':[]}

for i,seq in enumerate(libtest.population):
    name=f'TEST{i+1}'
    helm = ".".join(list(seq))
    mol = Chem.MolFromHELM("PEPTIDE1{%s}$$$$" % helm)
    mw = Descriptors.MolWt(mol)

    dict_comparison['ID'].append(name)
    dict_comparison['Sequence'].append(seq)
    dict_comparison['MW'].append(mw)

df = pd.DataFrame(dict_comparison)
df  

,ID,Sequence,MW
0,TEST1,FYCREWCWN,1306.496
1,TEST2,FACREWCWN,1214.399
2,TEST3,FACREWMWN,1242.453
3,TEST4,FNCRFWCWN,1275.486
4,TEST5,FYCREWVWN,1302.483
...,...,...,...
95,TEST96,FATRFWMWN,1258.474
96,TEST97,FNGRKWVWN,1206.377
97,TEST98,FNSRRWFWN,1312.461
98,TEST99,FIQRLWFWN,1309.541


Then we import the MatchedPairs class, and define the column(s) containing the property, the sequence, and the one having the id. Finally, we provide the reference peptide that will be taken into account to do the numbering of the mutations

In [9]:
# Import pandas
from pepfunn.pairs import MatchedPairs
property_columns = ['MW']
id_column='ID'
seq_column='Sequence'

# Reference substrate
reference='FNCREWCWN'

Here we call the first function to extract the region in the sequence where the substrate is, plus some additional columns explained below:

In [11]:
df_sequences = MatchedPairs.get_sequences(df, id_column=id_column, seq_column=seq_column, seq_ref=reference)
df_sequences

,ID,Begin,End,Peptide,REF
0,TEST1,,,FYCREWCWN,FYCREWCWN
1,TEST2,,,FACREWCWN,FACREWCWN
2,TEST3,,,FACREWMWN,FACREWMWN
3,TEST4,,,FNCRFWCWN,FNCRFWCWN
4,TEST5,,,FYCREWVWN,FYCREWVWN
...,...,...,...,...,...
95,TEST96,,,FATRFWMWN,FATRFWMWN
96,TEST97,,,FNGRKWVWN,FNGRKWVWN
97,TEST98,,,FNSRRWFWN,FNSRRWFWN
98,TEST99,,,FIQRLWFWN,FIQRLWFWN


The generated dataframe contains the following:
- **ID:** id assigned to each molecule in the library
- **Begin:** The sequence fragment before the start of the substrate sequence
- **End:** The sequence fragment after the end of the substrate sequence
- **Peptide:** Original peptide sequence
- **REF (it can be changed with the `name_ref` parameter):** This will contain the fragment that will be compared. The numbering will be defined based on this reference.

After this, we can calculate all the pairs. We will use the `df_sequences` dataframe to assign all the possible mutations:

In [12]:
df_pairs = MatchedPairs.get_pairs(df, df_sequences, property_columns, operation_columns=['divide'])
df_pairs

,ID_A,ID_B,Mutations,Distance,Diff_MW,Min_MW,Max_MW
0,TEST1,TEST2,(2)Y/A,1,1.075838,1214.399,1306.496
1,TEST1,TEST3,(2)Y/A | (7)C/M,2,1.051546,1242.453,1306.496
2,TEST1,TEST4,(2)Y/N | (5)E/F,2,1.024312,1275.486,1306.496
3,TEST1,TEST5,(7)C/V,1,1.003081,1302.483,1306.496
4,TEST1,TEST6,(2)Y/A | (5)E/F,2,1.060071,1232.461,1306.496
...,...,...,...,...,...,...,...
9895,TEST99,TEST97,(2)I/N | (3)Q/G | (5)L/K | (7)F/V,4,1.085516,1206.377,1309.541
9896,TEST100,TEST97,(2)A/N | (3)C/G | (5)E/K,3,1.003323,1206.377,1210.386
9897,TEST99,TEST98,(2)I/N | (3)Q/S | (5)L/R,3,0.997775,1309.541,1312.461
9898,TEST100,TEST98,(2)A/N | (3)C/S | (5)E/R | (7)V/F,4,0.922226,1210.386,1312.461


From the 100 sequences, a total of 9900 matched pairs were detected with directionality A->B. The `operation_columns` variable will describe how the experimental values will be compared, using either substract or divide operations (by default they will be divided). The following are the generated columns:

- **ID_A:** id of peptide A
- **ID_A:** id of peptide B
- **Mutations:** List of mutations found in the reference fragments following the format `(1)C/D | (2)E/F`, where the number in parenthesis is the position based on the reference, and the mutation is replacing `C` by `D` or `E` by `F`. If multiple mutations are reported, they will be separated by `|` symbol.
- **Distance:** The Hamming distance (number of mutations) between the fragments containing the reference peptide.
- **Diff, Min and Max columns per property:** These contain the difference of the assay/property based on the operation. This can be used to decide which mutations are improving or affecting negatively the property. The Min and Max columns are also reported to filter based on erroneous or not desired activity/property values

The `df_pairs` can be filtered even further depending on the question to solve. For example check cases where a number of mutations improve the activity towards a receptor, or focus on what mutations are decreasing the activity in an experimental assays. The way to analyze the data is open to the researcher and project.

### 2. Using a dataset without a reference peptide

Here we will use a similar example but without including a reference peptide. This allow running alignments that can create gaps between the sequences. This is usual for cases with sequences having different lengths, or when no specific reference is available

In [15]:
id_column='ID'
seq_column='Sequence'
property_columns=['MW']

df_sequences = MatchedPairs.get_sequences_simple(df, id_column, seq_column)
df_sequences

,ID,Peptide
0,TEST1,FYCREWCWN
1,TEST2,FACREWCWN
2,TEST3,FACREWMWN
3,TEST4,FNCRFWCWN
4,TEST5,FYCREWVWN
...,...,...
95,TEST96,FATRFWMWN
96,TEST97,FNGRKWVWN
97,TEST98,FNSRRWFWN
98,TEST99,FIQRLWFWN


To calculate the pairs we can use another function that will incorporate the gaps between each pair of sequences:

In [16]:
import warnings
warnings.filterwarnings('ignore')

df_pairs = MatchedPairs.get_pairs_simple(df, df_sequences, property_columns, operation_columns='')
df_pairs

,ID_A,ID_B,Mutations,Diff_MW,Min_MW,Max_MW
0,TEST1,TEST2,(2)Y/(2)A,1.075838,1214.399,1306.496
1,TEST1,TEST3,(2)Y/(2)A | (7)C/(7)M,1.051546,1242.453,1306.496
2,TEST1,TEST4,(2)Y/(2)N | (5)E/(5)F,1.024312,1275.486,1306.496
3,TEST1,TEST5,(7)C/(7)V,1.003081,1302.483,1306.496
4,TEST1,TEST6,(2)Y/(2)A | (5)E/(5)F,1.060071,1232.461,1306.496
...,...,...,...,...,...,...
9895,TEST99,TEST97,(2)I/(0)- | (3)Q/(3)N | (0)-/(4)G | (6)L/(6)K ...,1.085516,1206.377,1309.541
9896,TEST100,TEST97,(0)-/(2)N | (3)A/(3)G | (4)C/(0)- | (6)E/(6)K,1.003323,1206.377,1210.386
9897,TEST99,TEST98,(2)I/(2)N | (3)Q/(3)S | (5)L/(5)R,0.997775,1309.541,1312.461
9898,TEST100,TEST98,(2)A/(2)N | (3)C/(3)S | (5)E/(5)R | (7)V/(7)F,0.922226,1210.386,1312.461


In the results we will have similar columns to the first case, but now with the inclusion of gaps on the mutations list.

For any questions, please contact: raoc@novonordisk.com